In [62]:
from pyspark.sql import SparkSession, Row
from pyspark import SparkConf, SparkContext

spark = SparkSession.builder.master("local").appName("spark-dataframe-basic").getOrCreate()

In [63]:
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark.sql.window import Window

df = spark.read.load("./archive (1)/2019-Dec.csv",
                    format = "csv",
                    inferSchema = "true",
                    header = "true")

df.show()

+-------------------+----------------+----------+-------------------+-------------+---------+-----+---------+--------------------+
|         event_time|      event_type|product_id|        category_id|category_code|    brand|price|  user_id|        user_session|
+-------------------+----------------+----------+-------------------+-------------+---------+-----+---------+--------------------+
|2019-12-01 09:00:00|remove_from_cart|   5712790|1487580005268456287|         null|    f.o.x| 6.27|576802932|51d85cb0-897f-48d...|
|2019-12-01 09:00:00|            view|   5764655|1487580005411062629|         null|      cnd|29.05|412120092|8adff31e-2051-489...|
|2019-12-01 09:00:02|            cart|      4958|1487580009471148064|         null|   runail| 1.19|494077766|c99a50e8-2fac-4c4...|
|2019-12-01 09:00:05|            view|   5848413|1487580007675986893|         null|freedecor| 0.79|348405118|722ffea5-73c0-492...|
|2019-12-01 09:00:07|            view|   5824148|1487580005511725929|         null|

In [64]:
df.count()

3533286

In [65]:
df.printSchema()

root
 |-- event_time: timestamp (nullable = true)
 |-- event_type: string (nullable = true)
 |-- product_id: integer (nullable = true)
 |-- category_id: long (nullable = true)
 |-- category_code: string (nullable = true)
 |-- brand: string (nullable = true)
 |-- price: double (nullable = true)
 |-- user_id: integer (nullable = true)
 |-- user_session: string (nullable = true)



In [66]:
df.agg(max("event_time"), min("event_time")).show(truncate=False)

+-------------------+-------------------+
|max(event_time)    |min(event_time)    |
+-------------------+-------------------+
|2020-01-01 08:59:57|2019-12-01 09:00:00|
+-------------------+-------------------+



In [67]:
df.groupBy(col("brand"), col("category_id")).agg(count("*")).show()

+---------+-------------------+--------+
|    brand|        category_id|count(1)|
+---------+-------------------+--------+
|    domix|1487580011970953351|     963|
|    domix|1487580007256556476|     828|
|  bioaqua|1597770225539875791|     141|
|   runail|1487580007936033754|    3436|
|     oniq|1487580005092295511|    1778|
| ingarden|1487580011996119176|     970|
|  markell|1783999067156644376|     138|
|    naomi|1487580012524601496|      89|
|   missha|1783999073758478650|       4|
|     null|1487580008145748965|   15697|
|     null|1487580011677352062|    5306|
|     null|1998040852064109417|     808|
|    estel|1487580013732561106|    2191|
|   eunyul|1487580011585077370|    1882|
|   matrix|1487580008263189483|     518|
|   runail|1487580009051717646|    2417|
|      pnb|1487580007457883075|     409|
|     kiss|1487580013506068678|     195|
|   matrix|1487580010268065854|      60|
|veraclara|1783999072332415142|     214|
+---------+-------------------+--------+
only showing top

In [68]:
df.groupBy(col("brand"), col("category_code")).agg(countDistinct("product_id").alias("product_count")).show()

+------------+--------------------+-------------+
|       brand|       category_code|product_count|
+------------+--------------------+-------------+
|     beautix|                null|          324|
|  dr.gloderm|                null|           29|
|     farmona|                null|           38|
|      runail|appliances.enviro...|            3|
|   profhenna|                null|           54|
|     philips|                null|            1|
|invisibobble|                null|            3|
|       riche|                null|           36|
|        oniq|                null|          565|
|    lebelage|                null|           44|
|     vilenta|                null|           20|
|       fancy|                null|           14|
|    siberina|                null|          185|
|      tertio|                null|           98|
|      jaguar|                null|           21|
|    nitrimax|       apparel.glove|           19|
|   koreatida|                null|            3|


In [69]:
df.select("event_type").distinct().show()

+----------------+
|      event_type|
+----------------+
|        purchase|
|            view|
|            cart|
|remove_from_cart|
+----------------+



In [70]:
df\
    .selectExpr("to_date(event_time) as event_date", "event_type")\
    .where(col("event_type") == lit("purchase"))\
    .groupBy(col("event_date"))\
    .agg(count(col("event_type")).alias("purchase_count"))\
    .orderBy(asc(col("event_date")))\
    .show(truncate=False)

+----------+--------------+
|event_date|purchase_count|
+----------+--------------+
|2019-12-01|3938          |
|2019-12-02|8719          |
|2019-12-03|8949          |
|2019-12-04|7941          |
|2019-12-05|8222          |
|2019-12-06|7608          |
|2019-12-07|6124          |
|2019-12-08|6185          |
|2019-12-09|8280          |
|2019-12-10|10105         |
|2019-12-11|9205          |
|2019-12-12|10016         |
|2019-12-13|8163          |
|2019-12-14|6274          |
|2019-12-15|6597          |
|2019-12-16|8883          |
|2019-12-17|8383          |
|2019-12-18|8243          |
|2019-12-19|7624          |
|2019-12-20|6544          |
+----------+--------------+
only showing top 20 rows



In [71]:
df\
    .select("brand", "price")\
    .groupBy("brand")\
    .agg(sum("price").alias("sales_price"))\
    .orderBy(desc("sales_price"))\
    .limit(10)\
    .show()

+--------+--------------------+
|   brand|         sales_price|
+--------+--------------------+
|    null|1.1286279439998714E7|
|  strong|   2102395.009999969|
|jessnail|   2054767.029999821|
|  runail|   1716416.720000293|
|   irisk|  1169752.2900000443|
|marathon|   868768.1700000158|
| grattol|    856684.059999618|
|     cnd|    572103.229999986|
|   estel|   513535.1599999732|
|  masura|  425557.81999999704|
+--------+--------------------+



In [79]:
"""
ARPU (Average Revenue Per User): 전체 사용자 대비 수익입니다.

ARPPU (Average Revenue Per Paid User): 결제 사용자 대비 수익입니다.
"""
eventPurchase = "purchase"

df\
    .select("brand", "category_code", "price", "user_id", "event_type")\
    .where("brand IS NOT NULL")\
    .groupBy("brand")\
    .agg(
        countDistinct("user_id").alias("user_count_all"),
        countDistinct(when(col("event_type") == lit(eventPurchase), col("user_id"))).alias("user_count_purchase"),
        sum("price").alias("sales_price")
        )\
    .selectExpr(
        "brand", 
        "sales_price", 
        "user_count_all", 
        "sales_price / user_count_all as ARPU",
        "user_count_purchase",
        "sales_price / user_count_purchase as ARPPU",
    )\
    .orderBy(desc("sales_price"))\
    .limit(10)\
    .show(truncate=False)


+--------+------------------+--------------+------------------+-------------------+------------------+
|brand   |sales_price       |user_count_all|ARPU              |user_count_purchase|ARPPU             |
+--------+------------------+--------------+------------------+-------------------+------------------+
|strong  |2102395.009999969 |4659          |451.25456321098284|110                |19112.68190909063 |
|jessnail|2054767.029999821 |19018         |108.04327636974556|1073               |1914.973932898249 |
|runail  |1716416.720000293 |51516         |33.31812873670885 |7443               |230.6081848717309 |
|irisk   |1169752.2900000443|39894         |29.321509249512314|5864               |199.48026773534178|
|marathon|868768.1700000158 |3145          |276.2378918918969 |90                 |9652.979666666843 |
|grattol |856684.059999618  |31705         |27.0204718498539  |3674               |233.17475775710886|
|cnd     |572103.229999986  |8626          |66.32311963830118 |652       

In [91]:
# 전체 기간동안 브랜드별로 두번째로 많이 팔린 (판매 금액 총합이 높은) 상품 카테고리는 무엇입니까?
# SQL 사용
df.createOrReplaceTempView("PURCHASE")

spark.sql("""
        WITH CALCULATED AS(
            SELECT
                brand,
                category_code,
                sum(price) as sum_price
            FROM
                PURCHASE
            WHERE
                brand IS NOT NULL
            AND category_code IS NOT NULL
            GROUP BY brand, category_code
        ),
        CAL_RANKED AS(
            SELECT
                brand,
                category_code,
                ROW_NUMBER() OVER(PARTITION BY brand ORDER BY sum_price DESC) as rank
            FROM
                CALCULATED
                )
        SELECT * 
        FROM
            CAL_RANKED
        WHERE rank = 1
""").show()

+---------+--------------------+----+
|    brand|       category_code|rank|
+---------+--------------------+----+
| babyliss|appliances.person...|   1|
|   benovy|       apparel.glove|   1|
|  concept|appliances.enviro...|   1|
|   cutrin|appliances.enviro...|   1|
|depilflax| stationery.cartrige|   1|
|    dewal|appliances.person...|   1|
|    domix|furniture.bathroo...|   1|
|     emil|appliances.enviro...|   1|
|   entity|furniture.bathroo...|   1|
|    estel|appliances.enviro...|   1|
|farmavita|appliances.enviro...|   1|
|  farmona|furniture.bathroo...|   1|
|   gehwol|furniture.bathroo...|   1|
|    irisk|furniture.bathroo...|   1|
|  italwax| stationery.cartrige|   1|
|   jaguar|appliances.person...|   1|
| jessnail|appliances.enviro...|   1|
|   kaaral|appliances.enviro...|   1|
| kinetics|furniture.bathroo...|   1|
| kosmekka|furniture.living_...|   1|
+---------+--------------------+----+
only showing top 20 rows



In [112]:
# 전체 기간동안 브랜드별로 두번째로 많이 팔린 (판매 금액 총합이 높은) 상품 카테고리는 무엇입니까?
# DataFrame API 사용
dfCalculated = df\
    .select("brand", "category_id", "price")\
    .where(col("brand").isNotNull() & col("category_id").isNotNull())\
    .groupBy(col("brand"), col("category_id"))\
    .agg(sum(col("price")).alias("sum_price"))

dfRanked = dfCalculated\
    .select(
        col("brand"),
        col("category_id"),
        rank().over(Window.partitionBy(col("brand")).orderBy(desc("sum_price"))).alias("rank")
    )\
    .where(col("rank") == lit(1))\
    .show(truncate=False)

+-------------+-------------------+----+
|brand        |category_id        |rank|
+-------------+-------------------+----+
|airnails     |1487580013950664926|1   |
|almea        |1487580013522845895|1   |
|andrea       |2089259162625114209|1   |
|ardell       |2089259162625114209|1   |
|art-visage   |1487580013472514244|1   |
|artex        |1487580005092295511|1   |
|aura         |2151191059827262021|1   |
|australis    |1487580013413793985|1   |
|avene        |1783999072332415142|1   |
|babyliss     |1487580008053474272|1   |
|balbcare     |1487580009672474664|1   |
|barbie       |1511892746070131099|1   |
|barex        |1487580008246412266|1   |
|batiste      |1645114480121610699|1   |
|beautific    |2130081478220972046|1   |
|beautix      |1487580004832248652|1   |
|beauty-free  |1842735758805303837|1   |
|beautyblender|1487580013430571202|1   |
|beauugreen   |1891434351850160381|1   |
|benovy       |2007399943458784057|1   |
+-------------+-------------------+----+
only showing top

In [127]:
# 일별로 많이 팔린 (판매 금액 총합이 높은) 브랜드별 랭킹 Top 3 는 무엇입니까?
# SQL 사용
df.createOrReplaceTempView("PURCHASE")

spark.sql("""
    WITH CALC_DATE as(
    SELECT 
        TO_DATE(event_time) as event_date,
        brand,
        sum(price) as sum_price
    FROM PURCHASE
    WHERE brand IS NOT NULL
    GROUP BY event_date, brand
    ),
    CALC_RANK as (
    SELECT
        event_date,
        brand,
        row_number() over(partition by event_date order by sum_price) as rank
    FROM
        CALC_DATE
    )

    SELECT
        *
    FROM
        CALC_RANK
    WHERE rank <= 3
    ORDER BY event_date, rank
""").show(truncate=False)

+----------+------------+----+
|event_date|brand       |rank|
+----------+------------+----+
|2019-12-01|tazol       |1   |
|2019-12-01|bodipure    |2   |
|2019-12-01|veraclara   |3   |
|2019-12-02|voesh       |1   |
|2019-12-02|rocknailstar|2   |
|2019-12-02|weaver      |3   |
|2019-12-03|barbie      |1   |
|2019-12-03|frozen      |2   |
|2019-12-03|weaver      |3   |
|2019-12-04|rocknailstar|1   |
|2019-12-04|uralsoap    |2   |
|2019-12-04|skinity     |3   |
|2019-12-05|uralsoap    |1   |
|2019-12-05|skinity     |2   |
|2019-12-05|ovale       |3   |
|2019-12-06|candy       |1   |
|2019-12-06|cosima      |2   |
|2019-12-06|rocknailstar|3   |
|2019-12-07|supertan    |1   |
|2019-12-07|cosima      |2   |
+----------+------------+----+
only showing top 20 rows



In [146]:
# 전체 기간동안 브랜드별 매출(판매 금액의 합) 을 구하되, 자신보다 한단계 높은 순위 또는 낮은 순위의 매출도 같이 표시하기
# SQL 사용
df.createOrReplaceTempView("PURCHASE")

spark.sql("""
    WITH CALC as (SELECT
        brand,
        sum(price) as sum_price
    FROM PURCHASE
    WHERE brand IS NOT NULL
    GROUP BY brand
    )
    SELECT
        brand,
        lag(sum_price, 1) OVER(ORDER BY sum_price DESC) as price_sum_prev,
        sum_price as price_sum_current,
        lead(sum_price, 1) OVER(ORDER BY sum_price DESC) as price_sum_next,
        ROW_NUMBER() OVER(ORDER BY sum_price DESC) as rank
    FROM
        CALC
    ORDER BY rank
""").show()

24/08/12 11:01:12 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/08/12 11:01:12 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/08/12 11:01:12 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/08/12 11:01:18 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/08/12 11:01:18 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/08/12 11:01:19 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/08/12 1

+---------+------------------+------------------+------------------+----+
|    brand|    price_sum_prev| price_sum_current|    price_sum_next|rank|
+---------+------------------+------------------+------------------+----+
|   strong|              null| 2102395.009999969| 2054767.029999821|   1|
| jessnail| 2102395.009999969| 2054767.029999821| 1716416.720000293|   2|
|   runail| 2054767.029999821| 1716416.720000293|1169752.2900000443|   3|
|    irisk| 1716416.720000293|1169752.2900000443| 868768.1700000158|   4|
| marathon|1169752.2900000443| 868768.1700000158|  856684.059999618|   5|
|  grattol| 868768.1700000158|  856684.059999618|  572103.229999986|   6|
|      cnd|  856684.059999618|  572103.229999986| 513535.1599999732|   7|
|    estel|  572103.229999986| 513535.1599999732|425557.81999999704|   8|
|   masura| 513535.1599999732|425557.81999999704|421874.90000001737|   9|
|      uno|425557.81999999704|421874.90000001737| 419586.7600000071|  10|
|      max|421874.90000001737| 419586.

In [152]:
# 일별로 모든 브랜드를 통틀어, 판매 금액의 합산을 누적으로 구하면 매출의 변화량은 어떤지 살펴보기
# SQL 사용
df.createOrReplaceTempView("PURCHASE")

spark.sql("""
    WITH CALCULATED AS(SELECT
        TO_DATE(event_time) as event_date,
        sum(price) as price_sum
    FROM
        PURCHASE
    WHERE brand IS NOT NULL
    GROUP BY event_date)
    SELECT
        event_date,
        price_sum,
        sum(price_sum) over(order by event_date RANGE BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) as price_acc
    FROM
        CALCULATED
    ORDER BY event_date
""").show()

24/08/12 11:26:44 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/08/12 11:26:44 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/08/12 11:26:44 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/08/12 11:26:56 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/08/12 11:26:56 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/08/12 11:26:57 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/08/12 1

+----------+-----------------+--------------------+
|event_date|        price_sum|           price_acc|
+----------+-----------------+--------------------+
|2019-12-01|379024.1699999943|   379024.1699999943|
|2019-12-02|770094.4599999458|    1149118.62999994|
|2019-12-03|781595.0399999327|  1930713.6699998728|
|2019-12-04|787531.4799999163|   2718245.149999789|
|2019-12-05|758357.1399999415|  3476602.2899997304|
|2019-12-06|699547.9699999529|   4176150.259999683|
|2019-12-07|610060.4899999627|   4786210.749999646|
|2019-12-08|645167.8099999959|   5431378.559999642|
|2019-12-09|779006.3599999349|   6210384.919999577|
|2019-12-10|795046.2999999389|   7005431.219999516|
|2019-12-11|742134.6999999186|   7747565.919999435|
|2019-12-12|728599.1099999169|   8476165.029999351|
|2019-12-13|652097.3199999349|   9128262.349999286|
|2019-12-14|578012.6099999499|   9706274.959999235|
|2019-12-15|666701.7899999392|1.0372976749999175E7|
|2019-12-16|791329.3899999056| 1.116430613999908E7|
|2019-12-17|

### Array Type

In [171]:
df\
    .where(col("brand").isNotNull())\
    .groupBy(col("brand"))\
    .agg(
        collect_list("product_id").alias("product_id_list"),
        size(collect_list("product_id")).alias("product_id_list_count"),
        collect_set("product_id").alias("product_id_set"),
        size(collect_set("product_id")).alias("product_id_set_count")
        
    )\
    .show()

+-------------+--------------------+---------------------+--------------------+--------------------+
|        brand|     product_id_list|product_id_list_count|      product_id_set|product_id_set_count|
+-------------+--------------------+---------------------+--------------------+--------------------+
|     airnails|[5699414, 5659900...|                14430|[5659670, 5870652...|                 195|
|        almea|[5859284, 5859276...|                  826|[5859299, 5859270...|                  32|
|       andrea|[5525802, 5525802...|                   90|  [5525787, 5525802]|                   2|
|       ardell|[5808666, 5685404...|                 4969|[5685395, 5808665...|                  55|
|   art-visage|[5776130, 5776130...|                10704|[5698281, 5812113...|                  68|
|        artex|[5857683, 5857677...|                 4144|[5857672, 5857673...|                  16|
|         aura|[5894297, 5894298...|                  220|[5894298, 5894295...|            

In [191]:
df\
    .selectExpr("to_date(event_time) as event_date", "brand", "product_id", "ARRAY(category_code, category_id) as category")\
    .where(col("brand").isNotNull()&col("category_code").isNotNull())\
    .groupBy(col("event_date"))\
    .agg(
        collect_set("product_id").alias("product_id_set"),
        collect_set("category").alias("category_set")
    )\
    .limit(1)\
    .show()

+----------+--------------------+--------------------+
|event_date|      product_id_set|        category_set|
+----------+--------------------+--------------------+
|2019-12-10|[5911195, 5889693...|[[appliances.envi...|
+----------+--------------------+--------------------+



24/08/12 13:55:50 ERROR RetryingBlockTransferor: Exception while beginning fetch of 1 outstanding blocks (after 3 retries)
java.io.IOException: Failed to connect to /192.168.0.10:56279
	at org.apache.spark.network.client.TransportClientFactory.createClient(TransportClientFactory.java:284)
	at org.apache.spark.network.client.TransportClientFactory.createClient(TransportClientFactory.java:214)
	at org.apache.spark.network.netty.NettyBlockTransferService$$anon$2.createAndStart(NettyBlockTransferService.scala:130)
	at org.apache.spark.network.shuffle.RetryingBlockTransferor.transferAllOutstanding(RetryingBlockTransferor.java:173)
	at org.apache.spark.network.shuffle.RetryingBlockTransferor.lambda$initiateRetry$0(RetryingBlockTransferor.java:206)
	at java.base/java.util.concurrent.Executors$RunnableAdapter.call(Executors.java:515)
	at java.base/java.util.concurrent.FutureTask.run(FutureTask.java:264)
	at java.base/java.util.concurrent.ThreadPoolExecutor.runWorker(ThreadPoolExecutor.java:113

### JSON
- from_json : 문자열 컬럼과타입을 파라미터로 받아, JSON 타입으로 변경
- to_json : 컬럼을 받아, 타입이 있는 데이터 값을 JSON으로 변경

In [192]:
df.printSchema()

root
 |-- event_time: timestamp (nullable = true)
 |-- event_type: string (nullable = true)
 |-- product_id: integer (nullable = true)
 |-- category_id: long (nullable = true)
 |-- category_code: string (nullable = true)
 |-- brand: string (nullable = true)
 |-- price: double (nullable = true)
 |-- user_id: integer (nullable = true)
 |-- user_session: string (nullable = true)



In [194]:
# dropDuplicates : 중복 제거
dfPrepared = df\
    .select("brand", "product_id", "category_code", "category_id")\
    .dropDuplicates()

dfPrepared.show()

+---------+----------+-------------+-------------------+
|    brand|product_id|category_code|        category_id|
+---------+----------+-------------+-------------------+
|   runail|      4958|         null|1487580009471148064|
| jessnail|   5894907|         null|1487580010100293687|
|     null|   5904704|         null|1487580007675986893|
|     null|   5608703|         null|1487580005553668971|
|   dermal|   5863925|         null|1487580011585077370|
| ingarden|   5819231|         null|1487580005713052531|
|     null|   5840348|         null|1487580006317032337|
|     null|   5823493|         null|1487580004916134735|
|     null|   5796819|         null|1487580005092295511|
|swarovski|   5707753|         null|1487580011383750769|
|    irisk|   5729898|         null|1487580008145748965|
|     null|   5867623|         null|1487580004916134735|
|     null|   5908242|         null|1487580007675986893|
|     null|   5908202|         null|1487580007675986893|
|     null|   5853778|         

In [195]:
print(df.count())
print(dfPrepared.count())

3533286


46096


In [203]:
dfJson = dfPrepared\
    .where(col("category_code").isNotNull())\
    .withColumn("category", to_json(struct(col("category_id"), col("category_code"))))\

dfJson.show(truncate=False)

+--------+----------+-------------------------------+-------------------+-------------------------------------------------------------------------------------+
|brand   |product_id|category_code                  |category_id        |category                                                                             |
+--------+----------+-------------------------------+-------------------+-------------------------------------------------------------------------------------+
|null    |5756536   |apparel.glove                  |2007399943458784057|{"category_id":2007399943458784057,"category_code":"apparel.glove"}                  |
|emil    |5861764   |appliances.environment.vacuum  |1487580006350586771|{"category_id":1487580006350586771,"category_code":"appliances.environment.vacuum"}  |
|italwax |5775813   |stationery.cartrige            |1487580013053083824|{"category_id":1487580013053083824,"category_code":"stationery.cartrige"}            |
|benovy  |5911195   |apparel.glove      

In [205]:
dfCategory = dfJson.select("brand", "product_id", "category")
dfCategory.show(truncate=False)

+--------+----------+-------------------------------------------------------------------------------------+
|brand   |product_id|category                                                                             |
+--------+----------+-------------------------------------------------------------------------------------+
|null    |5756536   |{"category_id":2007399943458784057,"category_code":"apparel.glove"}                  |
|emil    |5861764   |{"category_id":1487580006350586771,"category_code":"appliances.environment.vacuum"}  |
|italwax |5775813   |{"category_id":1487580013053083824,"category_code":"stationery.cartrige"}            |
|benovy  |5911195   |{"category_id":2007399943458784057,"category_code":"apparel.glove"}                  |
|null    |5856308   |{"category_id":2022622168218599898,"category_code":"furniture.living_room.chair"}    |
|null    |5910451   |{"category_id":1487580008070251489,"category_code":"appliances.personal.hair_cutter"}|
|max     |5855509   |{"categ

In [209]:
# get_json_object : json -> object
dfRecovered = dfCategory\
    .withColumn("category_id", get_json_object(col("category"), "$.category_id").cast(LongType()))\
    .withColumn("category_code", get_json_object(col("category"), "$.category_code"))

dfRecovered.show(truncate=False)

+--------+----------+-------------------------------------------------------------------------------------+-------------------+-------------------------------+
|brand   |product_id|category                                                                             |category_id        |category_code                  |
+--------+----------+-------------------------------------------------------------------------------------+-------------------+-------------------------------+
|null    |5756536   |{"category_id":2007399943458784057,"category_code":"apparel.glove"}                  |2007399943458784057|apparel.glove                  |
|emil    |5861764   |{"category_id":1487580006350586771,"category_code":"appliances.environment.vacuum"}  |1487580006350586771|appliances.environment.vacuum  |
|italwax |5775813   |{"category_id":1487580013053083824,"category_code":"stationery.cartrige"}            |1487580013053083824|stationery.cartrige            |
|benovy  |5911195   |{"category_id":2007

In [212]:
structCategory = StructType(
    [
        StructField("category_id", LongType(), True),
        StructField("category_code", StringType(), True)
    ]
)


dfStructed = dfCategory\
    .withColumn("category_parse", from_json(col("category"), structCategory))

dfStructed.show(truncate=False)

+--------+----------+-------------------------------------------------------------------------------------+------------------------------------------------------+
|brand   |product_id|category                                                                             |category_parse                                        |
+--------+----------+-------------------------------------------------------------------------------------+------------------------------------------------------+
|null    |5756536   |{"category_id":2007399943458784057,"category_code":"apparel.glove"}                  |{2007399943458784057, apparel.glove}                  |
|emil    |5861764   |{"category_id":1487580006350586771,"category_code":"appliances.environment.vacuum"}  |{1487580006350586771, appliances.environment.vacuum}  |
|italwax |5775813   |{"category_id":1487580013053083824,"category_code":"stationery.cartrige"}            |{1487580013053083824, stationery.cartrige}            |
|benovy  |5911195   |{